In [11]:
!pip install tensorflow-text

In [13]:
from pathlib import Path,PureWindowsPath,PurePosixPath
import re
import csv
import io

import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text

In [2]:
input_file  = "QueryResults.csv"
output_file = "stackOverFlow.csv"
Q_file      = "stackOverFlow_Q.csv"
path = PureWindowsPath(r'C:\Users\benjamin.bouchard\Documents\PERSONNEL\OCR\courseNLP\P5\final\data')
data = pd.read_csv(f'{path}/{Q_file}')

In [3]:
Qs = data["Questions"]

#### Bert Wording Embedding

In [21]:
Qs[7]

"How Do You Communicate Service Layer Messages/Errors to Higher Layers Using MVP?.  I'm currently writing an ASP.Net app from the UI down. I'm implementing an MVP architecture because I'm sick of Winforms and wanted something that had a better separation of concerns.  So with MVP, the Presenter handles events raised by the View. Here's some code that I have in place to deal with the creation of users:  [CSS+Lasso]  I have my main form validation done using the built in .Net Validation Controls, but now I need to verify that the data sufficiently satisfies the criteria for the Service Layer.  Let's say the following Service Layer messages can show up:  E-mail account already exists (failure)  Refering user entered does not exist (failure)  Password length exceeds datastore allowed length (failure)  Member created successfully (success)  Let's also say that more rules will be in the service layer that the UI cannot anticipate.  Currently I'm having the service layer throw an exception if

In [23]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3"
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

text_test = [Qs[7]]
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed
#tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/4"


{'input_mask': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>,
 'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### Universal Sentence Encoder

In [8]:
Qs[5]

'What is the difference between #include <filename> and #include "filename"?.  In the C and C++ programming languages, what is the difference between using angle brackets and using quotes in an  [Tera Term macro]  statement, as follows?  [Arduino]  [C] '

##### Embedding as a function

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed([Qs[0]])

##### Embedding using a Keras model

In [9]:
embed_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")
inputs = tf.keras.Input(shape=tf.shape(''), dtype=tf.string)
outputs = embed_layer(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
embeddings = model.predict(tf.data.Dataset.from_tensor_slices(Qs.iloc[4:6]).batch(5))
print(embeddings.shape)
embeddings

1/1 [==============================] - 0s 415ms/step
(2, 512)


array([[-0.01788572,  0.00237409,  0.00673477, ...,  0.06602099,
        -0.02383039, -0.06340342],
       [-0.02748683, -0.04152428,  0.00301326, ...,  0.0246256 ,
         0.02267475, -0.05142068]], dtype=float32)

In [19]:
#8232713 49750645
#test = data[data['Id'] == 50895806]
#parser_list = html_parser.MyHTMLParser(filter= lambda x: False)
#parser_str = html_parser.MyHTMLParser(filter= lambda x: False,container=html_parser.MixContainer)
#res = test['Body'].apply(html_parser.do_parse,args=(parser_str,))
#res.iloc[0]
#from pygments.lexers import guess_lexer
#str = "XmlInclude"
#lexer = guess_lexer(str)
#lexer.name
test = data.iloc[1000:1100].copy()

In [12]:
# parse html content in Body column
def define_questions(data):
    # parse body
    data['Body-parsed'] = data['Body'].apply(html_parser.do_parse,args=(html_parser.MyHTMLParser(filter= lambda x: False,container=html_parser.MixContainer ),))
    # concatenate title plus parse body
    data['Questions'] = data[['Title','Body-parsed']].apply(lambda r: r['Title'] + '. ' + r['Body-parsed'],axis=1)

#with io.open(f'{path}/{file_questions}','w', encoding="utf8") as f:
#    _ = list(map(lambda r: f.write(r+'\n'),buffer))


In [20]:
define_questions(test)

In [22]:
#dataset = tf.data.TextLineDataset([f'{path}/{file_questions}'])

dataset = tf.data.Dataset.from_tensor_slices(test["Questions"])